# finalTable
## Table of Content

1. [Import packages](#packages)
2. [Import data](#data)
3. [Data Cleaning](#cleaning)
    1. [Cleaning damage data](#damage)
    2. [Cleaning contract data](#contract)
4. [Joining Damage and Contract](#join)
5. [Risk Score Definition](#riskscore)

## Import packages <a id="packages"></a>

In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

In [5]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import data <a id="data"></a>

In [6]:
contractPath = '..\data\contract_all.csv'
damagePath = '..\data\damage.csv'

contract_all = pd.read_csv(contractPath)
damage = pd.read_csv(damagePath)

# Data Cleaning <a id="cleaning"></a>

## Cleaning damage data <a id="damage"></a>

Drop these variables:
- The vast majority of data in ADDRESS_TYPE, DAMAGE_REPORTER, DATE_COMPLETION_DATE and CUMULEVENT are missing.
- Exclude variables with only one (or very few/not relevant) category: 'DEPARTMENT_SEGMENT', 'DEPARTMENT_RESPONSIBLE', 'DEPARTMENT_CUSTOMER_SEGMENT', 'BUSINESS_TYPE_MARK', 'DAMAGE_BRANCH_DESIGNMENT', 'DAMAGE_BRANCH', 'SVCLUSTER', ‘BUSINESS_MARK’
- Not relevant: 'PARTY_ID', 'RESTORATION_CANDIDAT'

In [7]:
# Drop columns
columnsToDrop = ['DEPARTMENT_SEGMENT', 'DEPARTMENT_RESPONSIBLE', 'DEPARTMENT_CUSTOMER_SEGMENT', 'BUSINESS_TYPE_MARK', 'BUSINESS_MARK',
                 'DAMAGE_BRANCH_DESIGNMENT', 'DAMAGE_BRANCH', 'SVCLUSTER', 'ADDRESS_TYPE', 'DAMAGE_REPORTER', 'DATE_COMPLETION_DATE', 'PARTY_ID',
                 'CUMULEVENT', 'RESTORATION_CANDIDAT']

damage = damage.drop(columns=columnsToDrop, errors='ignore')

In [8]:
# Convert dates to date format
damage['DATE_DAMAGE_EVENT'] = pd.to_datetime(damage['DATE_DAMAGE_EVENT'])
damage['DATE_INITIAL_RESERVATION'] = pd.to_datetime(damage['DATE_INITIAL_RESERVATION'], dayfirst=True)

# Convert 'TOTAL_DAMAGE' to float
damage['TOTAL_DAMAGE'] = damage['TOTAL_DAMAGE'].replace('UNDEFINED', np.nan).astype('float64')

# Convert ids to object
damage['BUSINESS_FIELD_ID'] = damage['BUSINESS_FIELD_ID'].astype('object')
damage['CONTRACT_ID'] = damage['CONTRACT_ID'].astype('object')
damage['DAMAGE_TRAIT'] = damage['DAMAGE_TRAIT'].astype('object')
damage['DAMAGE_CAUSE'] = damage['DAMAGE_CAUSE'].astype('object')
damage['DAMAGE_ID'] = damage['DAMAGE_ID'].astype('object')
damage['STATISTIC_NUMBER'] = damage['STATISTIC_NUMBER'].astype('object')

In [9]:
# Drop observations outside of Germany
damage = damage[damage['DAMAGE_NATION'] == 'D']

# Drop column 'DAMAGE_NATION'
damage = damage.drop(columns=['DAMAGE_NATION'], errors='ignore')

len(damage)

305783

In [10]:
# Drop observations before 2014
damage = damage[damage['DATE_DAMAGE_EVENT'].dt.year >= 2014]
damage = damage[damage['DATE_INITIAL_RESERVATION'].dt.year >= 2014]
len(damage)

247134

Drop negative values from financial variables:

In [11]:
# Check number of negative values in financial columns
financialColumns = ['EXPENSE', 'INITIAL_RESERVATION', 'TOTAL_DAMAGE', 'RESIDUAL_AMOUNT', 'PAYOUT']

# Create a dictionary
negativeCounts = {}

# Count negative values in each column
for column in financialColumns:
    negativeCounts[column] = (damage[column] < 0).sum()

print(negativeCounts)

{'EXPENSE': 230, 'INITIAL_RESERVATION': 0, 'TOTAL_DAMAGE': 0, 'RESIDUAL_AMOUNT': 334, 'PAYOUT': 89}


In [12]:
# Create a mask that is True for rows where all financial columns are >= 0 or NaN
mask = (damage[financialColumns] >= 0) | damage[financialColumns].isna()

# Use .all(axis=1) to ensure that we only drop rows where all conditions are False (i.e., where all are negative)
damage = damage[mask.all(axis=1)]
len(damage)

246567

In [13]:
# Convert zip codes starting with 'W' or 'O', or those not 5 characters long to NaN
damage.loc[damage['DAMAGE_EVENT_ZIP_CODE'].str.startswith(('W', 'O')) | 
           (damage['DAMAGE_EVENT_ZIP_CODE'].str.len() != 5), 'DAMAGE_EVENT_ZIP_CODE'] = np.nan

In [14]:
# Replace "Quartalsabrechnung" or "Monatsabrechnung" with NaN
damage['DAMAGE_EVENT_STREET'] = damage['DAMAGE_EVENT_STREET'].replace(['Quartalsabrechnung', 'Monatsabrechnung'], np.nan)

In [15]:
# Exclude rows where EXPENSE, PAYOUT, and RESIDUAL_AMOUNT are all zero
# damage = damage[(damage['EXPENSE'] != 0) | (damage['PAYOUT'] != 0) | (damage['RESIDUAL_AMOUNT'] != 0)]
# print(len(damage))

## Cleaning contract data <a id="contract"></a>

Drop these variables:
- The majority of data is missing for the variables BASEMENT, BUILDING_TYPE, NO_CLAIMS_SCALE, NO_CLAIMS_YEARS.
- Not relevant for our analysis: 'TYPE_OF_DEDUCTIBLE', 'DEDUCTIBLE_PERCENTAGE', 'DEDUCTIBLE_MIN', 'DEDUCTIBLE_MAX', 'INVENTORY_MANAGEMENT_SYSTEM', 'PARTY-ID', 'RISK_NATION', 'Day', 'Month'

In [16]:
# Drop columns from all dataframes
columnsToDrop = ['TYPE_OF_DEDUCTIBLE', 'DEDUCTIBLE_PERCENTAGE', 'DEDUCTIBLE_MIN', 'DEDUCTIBLE_MAX', 'INVENTORY_MANAGEMENT_SYSTEM',
                'BASEMENT', 'BUILDING_TYPE', 'NO_CLAIMS_SCALE', 'NO_CLAIMS_YEARS', 'RISK_NATION', 'PARTY-ID', 'Day', 'Month']

contract_all = contract_all.drop(columns=columnsToDrop, errors='ignore')

In [17]:
# Convert dates to date format
contract_all['KEYDATE'] = pd.to_datetime(contract_all['KEYDATE'])

# Convert ids to object
contract_all['CONTRACT_ID'] = contract_all['CONTRACT_ID'].astype('object')

# Replace '     .' with NaN in PREMIUM_AMOUNT
contract_all['PREMIUM_AMOUNT'] = contract_all['PREMIUM_AMOUNT'].replace('     .', np.nan)
# Convert PREMIUM_AMOUNT to float
contract_all['PREMIUM_AMOUNT'] = contract_all['PREMIUM_AMOUNT'].astype(float)

# Replace '     .' with NaN in SUM_INSURED
contract_all['SUM_INSURED'] = contract_all['SUM_INSURED'].replace('     .', np.nan)
# Convert SUM_INSURED to float
contract_all['SUM_INSURED'] = contract_all['SUM_INSURED'].astype(float)

Drop negative values from financial variables (there are none):

In [18]:
# Check number of negative values in financial columns
financialColumns = ['SUM_INSURED', 'PREMIUM_AMOUNT']

# Create a dictionary
negativeCounts = {}

# Count negative values in each column
for column in financialColumns:
    negativeCounts[column] = (contract_all[column] < 0).sum()

print(negativeCounts) # nothing to drop

{'SUM_INSURED': 0, 'PREMIUM_AMOUNT': 0}


In [19]:
# Convert zip codes starting with 'W' or 'O', or those not 5 characters long to NaN
contract_all.loc[contract_all['RISK_LOCATION_ZIP_CODE'].str.startswith(('W', 'O')) | 
            (contract_all['RISK_LOCATION_ZIP_CODE'].str.len() != 5), 'RISK_LOCATION_ZIP_CODE'] = np.nan

In [20]:
# Replace living space = 0 with average

# Calculate the average of LIVING_SPACE, excluding zeros
non_zero_living_space = contract_all[contract_all['LIVING_SPACE'] != 0]['LIVING_SPACE']
avg_living_space = non_zero_living_space.mean()

# Replace zero values with the calculated average
contract_all.loc[contract_all['LIVING_SPACE'] == 0, 'LIVING_SPACE'] = avg_living_space

In [21]:
# Replace uncommon values for the construction year with the average
currentYear = 2024

# Calculate the average of CONSTRUCTION_YEAR, excluding future years and specific unwanted values
validYears = contract_all[
    (contract_all['CONSTRUCTION_YEAR'] <= currentYear) &
    (~contract_all['CONSTRUCTION_YEAR'].isin([1111, 1, 0]))
]['CONSTRUCTION_YEAR']
avgYear = validYears.mean()

# Replace unwanted values with the calculated average
contract_all.loc[
    (contract_all['CONSTRUCTION_YEAR'] > currentYear) |
    (contract_all['CONSTRUCTION_YEAR'].isin([1111, 1, 0])), 'CONSTRUCTION_YEAR'
] = avgYear

In [22]:
# Replace values in the 'DRAIN_PIPE_INSURED' column
contract_all['DRAIN_PIPE_INSURED'] = contract_all['DRAIN_PIPE_INSURED'].replace({'J': 1, 'N': 0})

# Replace values in the 'UNDERINSURANCE_WAIVER' column
contract_all['UNDERINSURANCE_WAIVER'] = contract_all['UNDERINSURANCE_WAIVER'].replace({'J': 1, 'N': 0})

In [23]:
%store damage

Stored 'damage' (DataFrame)


In [24]:
%store contract_all

Stored 'contract_all' (DataFrame)


# Joining Damage and Contract <a id="join"></a>

In [25]:
# Extract year into new column (already done for contract table)
damage['Year'] = damage['DATE_INITIAL_RESERVATION'].dt.year

In [26]:
# Combine Damage and Contract Data
damage_all = pd.merge(damage, contract_all, left_on=["CONTRACT_ID", "Year"], right_on=["CONTRACT_ID", "Year"], how='left')
print(len(damage))
print(len(damage_all))

246567
251865


In [27]:
# Delete Rows that have been in both datasets
damage_all = damage_all.drop(["RISK_LOCATION_ZIP_CODE", "RISK_LOCATION_TOWN", "RISK_LOCATION_STREET"], axis=1)

In [28]:
damage_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251865 entries, 0 to 251864
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   EXPENSE                   251865 non-null  float64       
 1   DAMAGE_CLUSTER_ID         251865 non-null  object        
 2   DATE_INITIAL_RESERVATION  251865 non-null  datetime64[ns]
 3   DATE_DAMAGE_EVENT         251865 non-null  datetime64[ns]
 4   CONTRACT_ID               251865 non-null  object        
 5   INITIAL_RESERVATION       251865 non-null  float64       
 6   TOTAL_DAMAGE              199375 non-null  float64       
 7   BUSINESS_FIELD_ID         251865 non-null  object        
 8   RESIDUAL_AMOUNT           251865 non-null  float64       
 9   DAMAGE_TYPE               251865 non-null  object        
 10  DAMAGE_INCIDENT           251865 non-null  object        
 11  DAMAGE_EVENT_TOWN         251865 non-null  object        
 12  DA

# Risk Score Definition <a id="riskscore"></a>

In [29]:
# Calculate NET_PREMIUM_AMOUNT as 60% of PREMIUM_AMOUNT
damage_all['NET_PREMIUM_AMOUNT'] = damage_all['PREMIUM_AMOUNT'] * 0.6

In [30]:
# Calculate PIPE_PREMIUM_AMOUNT based on CORPORATE_DEVISION
# Map devision percentages
devisionPercentages = {
    'VGV': 0.43,
    'VHV': 0.11,
    'H&H': 0.07,
    'W&W': 0.32
}

# Calculate PIPE_PREMIUM_AMOUNT
damage_all['PIPE_PREMIUM_AMOUNT'] = damage_all['DIVISION_LEVEL_02'].map(devisionPercentages) * damage_all['NET_PREMIUM_AMOUNT']

In [31]:
# Calculate RISK_SCORE as PIPE_PREMIUM_AMOUNT / EXPENSE
damage_all['RISK_SCORE'] = damage_all['EXPENSE'] / damage_all['PIPE_PREMIUM_AMOUNT']

# When expense = 0 the division returns 0. Instead return NaN.
# damage_all['RISK_SCORE'] = damage_all['RISK_SCORE'].replace(0, np.nan)

In [32]:
# Check for nas in RISK_SCORE
print(len(damage_all))
print(damage_all['RISK_SCORE'].isna().sum())
print(len(damage_all.dropna(subset=['RISK_SCORE'])))

251865
32312
219553


In [33]:
# Check nas in PREMIUM_AMOUNT and EXPENSE
print(damage_all['PREMIUM_AMOUNT'].isna().sum())
print(damage_all['EXPENSE'].isna().sum())

31975
0


In [34]:
# Save dataset to csv
damage_all.to_csv('../data/damage_all.csv', index=False)

In [35]:
%store damage_all

Stored 'damage_all' (DataFrame)


In [36]:
# %store -r damage
# %store -r contract_all
# %store -r damage_all